# Indexing, Selection, and Operations

Now that we are familiar with pandas' data structures, we can turn our attention to some of the intermediate features of data frames, which include:
    
- Intelligent label-based **slicing, fancy indexing, and subsetting** of large data sets
- **Hierarchical labeling** of axes
- **Sorting and ranking** of data in DataFrames
- Easy handling of **missing data**
- Data **summarization** tools

In this section, we will manipulate data collected from ocean-going vessels on the eastern seaboard. Vessel operations are monitored using the **Automatic Identification System (AIS)**, a safety at sea navigation technology which vessels are required to maintain and that uses transponders to transmit very high frequency (VHF) radio signals containing static information including ship name, call sign, and country of origin, as well as dynamic information unique to a particular voyage such as vessel location, heading, and speed. 

![AIS](images/ais.gif)

The International Maritime Organization’s (IMO) International Convention for the Safety of Life at Sea requires functioning AIS capabilities on all vessels 300 gross tons or greater and the US Coast Guard requires AIS on nearly all vessels sailing in U.S. waters. The Coast Guard has established a national network of AIS receivers that provides coverage of nearly all U.S. waters. **AIS signals** are transmitted several times each minute and the network is capable of handling thousands of reports per minute and updates as often as every two seconds. Therefore, a typical voyage in our study might include the transmission of hundreds or thousands of AIS encoded signals. This provides a rich source of spatial data that includes both **spatial and temporal information**.

For our purposes, we will use **summarized data** that describes the transit of a given vessel through a particular administrative area. The data includes the start and end time of the transit segment, as well as information about the speed of the vessel, how far it travelled, etc.

In [1]:
import pandas as pd
import numpy as np

vessels = pd.read_csv('../data/AIS/vessel_information.csv', index_col=0)

## Indexing and Selection

Indexing works analogously to indexing in NumPy arrays, except we can use the labels in the `Index` object to extract values in addition to arrays of integers.

In [2]:
# Sample Series object
flag = vessels.flag
flag

mmsi
1                                               Unknown
9                                               Unknown
21                                              Unknown
74                                              Unknown
103                                             Unknown
310                                            Bermuda 
3011                                          Anguilla 
4731                               Yemen (Republic of) 
15151                                           Unknown
46809                             Syrian Arab Republic 
80404                                           Unknown
82003                                           Unknown
298716                                          Unknown
366235                        United States of America 
439541                                          Unknown
453556                                          Unknown
505843                                          Unknown
527918                                     

In [3]:
# Numpy-style indexing
flag[:10]

mmsi
1                      Unknown
9                      Unknown
21                     Unknown
74                     Unknown
103                    Unknown
310                   Bermuda 
3011                 Anguilla 
4731      Yemen (Republic of) 
15151                  Unknown
46809    Syrian Arab Republic 
Name: flag, dtype: object

In [4]:
# Indexing by label
flag[[298716,725011300]]

mmsi
298716       Unknown
725011300     Chile 
Name: flag, dtype: object

In a `DataFrame` we can slice along either or both axes:

In [6]:
vessels[['num_names','num_types']].head()

,num_names,num_types
mmsi,,
1,8,4
9,3,2
21,1,1
74,2,1
103,3,2


In [7]:
vessels[vessels.max_loa>700]

,num_names,names,sov,flag,flag_type,num_loas,loa,max_loa,num_types,type
mmsi,,,,,,,,,,
211311970,2,Antwerpen Express/Dallas Express,N,Germany (Federal Republic of),Foreign,3,294.0/512.0/806.0,806.0,1,Cargo
218292000,1,Seapike,N,Germany (Federal Republic of),Foreign,3,199.0/200.0/711.0,711.0,1,Tanker
235760000,1,Methane Princess,N,United Kingdom of Great Britain and Northern I...,Foreign,2,277.0/789.0,789.0,1,Tanker
354092000,1,Msc Laura,N,Panama (Republic of),Foreign,3,300.0/301.0/812.0,812.0,1,Cargo
538090124,1,Pacific Destiny,N,Marshall Islands (Republic of the),Foreign,2,196.0/708.0,708.0,1,Cargo


The indexing field `loc` allows us to select subsets of rows and columns in an intuitive way:

In [8]:
vessels.loc[720768000, ['names','flag', 'type']]

names             Rusty Pelican
flag     Bolivia (Republic of) 
type                     Towing
Name: 720768000, dtype: object

In [9]:
vessels.loc[:4731, 'names']

mmsi
1       Bil Holman Dredge/Dredge Capt Frank/Emo/Offsho...
9                              000000009/Raven/Shearwater
21                                          Us Gov Vessel
74                                      Mcfaul/Sarah Bell
103              Ron G/Us Navy Warship 103/Us Warship 103
310                                              Arabella
3011                                           Charleston
4731                                            000004731
Name: names, dtype: object

Slicing also works with string variables, since an index has an intrinsic order, regardless of label:

In [10]:
vessels.loc[:310, 'flag':'loa']

,flag,flag_type,num_loas,loa
mmsi,,,,
1,Unknown,Unknown,7,42.0/48.0/57.0/90.0/138.0/154.0/156.0
9,Unknown,Unknown,2,50.0/62.0
21,Unknown,Unknown,1,208.0
74,Unknown,Unknown,1,155.0
103,Unknown,Unknown,2,26.0/155.0
310,Bermuda,Foreign,1,47.0


In addition to using `loc` to select rows and columns by **label**, pandas also allows indexing by **position** using the `iloc` attribute.

So, we can query rows and columns by absolute position, rather than by name:

In [11]:
vessels.iloc[:5, 5:8]

,num_loas,loa,max_loa
mmsi,,,
1,7,42.0/48.0/57.0/90.0/138.0/154.0/156.0,156.0
9,2,50.0/62.0,62.0
21,1,208.0,208.0
74,1,155.0,155.0
103,2,26.0/155.0,155.0


### Exercise

You can use the `isin` method query a DataFrame based upon a list of values as follows: 

    data['color'].isin(['red', 'blue'])

Use `isin` to find all ships that registered in either Denmark or Japan. How many records contain these values?

In [12]:
# Write your answer here

## Indexing with `where`

Pandas `DataFrame` objects also posess a `where` index for indexing that returns the values that satisfy the condition, but retain the index of the original `DataFrame`, so that the shape does not change. This is important when **alignment** is required for operations between `DataFrame`s.

In [13]:
np.random.seed(42)
normal_vals = pd.DataFrame({'x{}'.format(i):np.random.randn(100) for i in range(5)})

normal_vals.head()

,x0,x1,x2,x3,x4
0,0.496714,-1.415371,0.357787,-0.828995,-1.594428
1,-0.138264,-0.420645,0.560785,-0.560181,-0.599375
2,0.647689,-0.342715,1.083051,0.747294,0.005244
3,1.523030,-0.802277,1.053802,0.610370,0.046981
4,-0.234153,-0.161286,-1.377669,-0.020902,-0.450065


In [14]:
normal_vals.where(normal_vals>0).head()

,x0,x1,x2,x3,x4
0,0.496714,NaN,0.357787,NaN,NaN
1,NaN,NaN,0.560785,NaN,NaN
2,0.647689,NaN,1.083051,0.747294,0.005244
3,1.523030,NaN,1.053802,0.610370,0.046981
4,NaN,NaN,NaN,NaN,NaN


`where` includes an optional `other` argument that accepts a scalar or tabular values (or a callable) to replace values in the `DataFrame` that do not satisfy the condition.

For example, we can use this to return the absolute values of `X`:

In [15]:
normal_vals.where(normal_vals>0, other=-normal_vals).head()

,x0,x1,x2,x3,x4
0,0.496714,1.415371,0.357787,0.828995,1.594428
1,0.138264,0.420645,0.560785,0.560181,0.599375
2,0.647689,0.342715,1.083051,0.747294,0.005244
3,1.523030,0.802277,1.053802,0.610370,0.046981
4,0.234153,0.161286,1.377669,0.020902,0.450065


Similarly, a callable can be used when we need to modify the replaced value:

In [16]:
normal_vals.where(normal_vals>0, other=lambda y: -y*100).head()

,x0,x1,x2,x3,x4
0,0.496714,141.537074,0.357787,82.899501,159.442766
1,13.826430,42.064532,0.560785,56.018104,59.937502
2,0.647689,34.271452,1.083051,0.747294,0.005244
3,1.523030,80.227727,1.053802,0.610370,0.046981
4,23.415337,16.128571,137.766937,2.090159,45.006547


Conversely, `mask` is the inverse boolean of `where`:

In [17]:
normal_vals.mask(normal_vals>0).head()

,x0,x1,x2,x3,x4
0,NaN,-1.415371,NaN,-0.828995,-1.594428
1,-0.138264,-0.420645,NaN,-0.560181,-0.599375
2,NaN,-0.342715,NaN,NaN,NaN
3,NaN,-0.802277,NaN,NaN,NaN
4,-0.234153,-0.161286,-1.377669,-0.020902,-0.450065


## Selection with `query`

At times, selection using indexing can be verbose because it requires repeated use of the `DataFrame` namespace.

In [19]:
normal_vals[(normal_vals.x1 > normal_vals.x0) & (normal_vals.x3 > normal_vals.x2)].head()

,x0,x1,x2,x3,x4
4,-0.234153,-0.161286,-1.377669,-0.020902,-0.450065
5,-0.234137,0.404051,-0.937825,0.117327,0.622850
6,1.579213,1.886186,0.515035,1.277665,-1.067620
8,-0.469474,0.257550,0.515048,0.547097,0.120296
13,-1.913280,2.463242,0.651391,0.813510,1.277677


For a more concise (and readable) syntax, we can use the new `query` method to perform selection on a `DataFrame`. Instead of having to type the fully-specified column, we can simply pass a string that describes what to select. The query above is then simply:

In [20]:
normal_vals.query('(x1 > x0) & (x3 > x2)').head()

,x0,x1,x2,x3,x4
4,-0.234153,-0.161286,-1.377669,-0.020902,-0.450065
5,-0.234137,0.404051,-0.937825,0.117327,0.622850
6,1.579213,1.886186,0.515035,1.277665,-1.067620
8,-0.469474,0.257550,0.515048,0.547097,0.120296
13,-1.913280,2.463242,0.651391,0.813510,1.277677


The `DataFrame.index` and `DataFrame.columns` are placed in the query namespace by default. If you want to refer to a variable in the current namespace, you can prefix the variable with `@`:

In [21]:
min_loa = 700

In [22]:
vessels.query('max_loa > @min_loa')

,num_names,names,sov,flag,flag_type,num_loas,loa,max_loa,num_types,type
mmsi,,,,,,,,,,
211311970,2,Antwerpen Express/Dallas Express,N,Germany (Federal Republic of),Foreign,3,294.0/512.0/806.0,806.0,1,Cargo
218292000,1,Seapike,N,Germany (Federal Republic of),Foreign,3,199.0/200.0/711.0,711.0,1,Tanker
235760000,1,Methane Princess,N,United Kingdom of Great Britain and Northern I...,Foreign,2,277.0/789.0,789.0,1,Tanker
354092000,1,Msc Laura,N,Panama (Republic of),Foreign,3,300.0/301.0/812.0,812.0,1,Cargo
538090124,1,Pacific Destiny,N,Marshall Islands (Republic of the),Foreign,2,196.0/708.0,708.0,1,Cargo


## Operations

`DataFrame` and `Series` objects allow for several operations to take place either on a single object, or between two or more objects.

For this section, we will import a complementary dataset of transits (voyages) of each vessel. This is a much larger dataset, since ships each have multiple transits over several years.

For example, we can perform arithmetic on the elements of two objects, such as combining distances travelled across years. First, let's artificially construct two Series, consisting of segment lengths travelled in 2009 and 2010:

In [23]:
segments = pd.read_csv('../data/AIS/transit_segments.csv', parse_dates=['st_time'])

Extract the year from the date (more on dates/times later!)

In [24]:
segments['year'] = segments.st_time.dt.year
segments.year.head()

0    2009
1    2009
2    2009
3    2009
4    2009
Name: year, dtype: int64

In [25]:
segments2009 = segments[segments.year==2009].copy()
lengths2009 = pd.Series({name: segments2009[segments2009.name==name].seg_length.sum() 
                         for name in segments2009.name.unique()})

In [27]:
segments2010 = segments[segments.year==2010].copy()
lengths2010 = pd.Series({name: segments2010[segments2010.name==name].seg_length.sum() 
                         for name in segments2010.name.unique()})

We will learn a much more efficient way of performing this operation when we get to `GroupBy` operations a little later on.

In [29]:
length2009 = segments.loc[segments.year==2009, 'seg_length']
length2009.index = segments.name[segments.year==2009]

length2010 = segments.loc[segments.year==2010, 'seg_length']
length2010.index = segments.name[segments.year==2010]

Now, let's add them together, in hopes of getting 2-year travel totals:

In [30]:
seg_lengths = lengths2009 + lengths2010
seg_lengths

$ Vega Azurit $            NaN
+ Alida Gorthon +          NaN
+alida Gorthon+           63.0
+margit Gorthon+           NaN
0mega Theodore             NaN
1700 Somewhere             NaN
1st Lt Alex Bonnyman       NaN
1st Lt Jack Lummus         NaN
205551000                  NaN
210235000                  NaN
210670000                 43.5
212043000                  5.2
212412000                  NaN
235067192                  NaN
235075024                  NaN
240716000                  NaN
245554000                  NaN
247102700                  NaN
247242400                  NaN
247281100                  NaN
247645000                 55.4
257356000                  NaN
304716000                  NaN
305624000                  NaN
311003300                  7.7
311003400                  NaN
311011100                  NaN
311011400                  NaN
311022500                  NaN
319015800                  NaN
                         ...  
Zim California          1166.4
Zim Haif

Pandas' data alignment places `NaN` values for labels that do not overlap in the two Series. In fact, the majority of ships only have data for one of the two years.

In [31]:
seg_lengths.notnull().mean()

0.35575589459084606

While we do want the operation to honor the data labels in this way, we probably do not want the missing values to be filled with `NaN`. We can use the `add` method to calculate ship travel totals by using the `fill_value` argument to insert a zero for years that do not have data for a particular vessel:

In [32]:
lengths2009.add(lengths2010, fill_value=0)

$ Vega Azurit $           69.0
+ Alida Gorthon +        497.2
+alida Gorthon+           63.0
+margit Gorthon+         342.3
0mega Theodore            62.6
1700 Somewhere             7.6
1st Lt Alex Bonnyman      96.4
1st Lt Jack Lummus         1.3
205551000                  7.2
210235000                  1.2
210670000                 43.5
212043000                  5.2
212412000                  4.9
235067192                 19.5
235075024                  1.1
240716000                 72.9
245554000                  4.1
247102700                 24.1
247242400                  8.9
247281100                 21.1
247645000                 55.4
257356000                135.3
304716000                  4.0
305624000                 53.6
311003300                  7.7
311003400                  4.3
311011100                 18.9
311011400                  3.3
311022500                 19.6
319015800                 10.7
                         ...  
Zim California          1166.4
Zim Haif

Operations can also be **broadcast** between rows or columns.

For example, if we subtract the maximum LOA (ship length) from the `max_loa` column, we get a the differences between the size of a given vessel to the largest vessel.

In [33]:
vessels.max_loa - vessels.max_loa.max()

mmsi
1           -656.0
9           -750.0
21          -604.0
74          -657.0
103         -657.0
310         -765.0
3011        -652.0
4731        -782.0
15151       -637.0
46809       -790.0
80404       -783.0
82003       -783.0
298716      -777.0
366235      -605.0
439541      -757.0
453556      -604.0
505843      -792.0
527918      -792.0
565026      -780.0
572329      -772.0
587370      -779.0
641114      -792.0
642262      -757.0
693559      -772.0
883085      -607.0
1193046     -143.0
1193946     -745.0
1233916     -792.0
1239468     -790.0
3041300     -694.0
             ...  
720728000   -714.0
720754000   -780.0
720768000   -783.0
725004700   -652.0
725005560   -635.0
725005570   -637.0
725011300   -645.0
725018300   -636.0
725019006   -699.0
725021000   -624.0
725022000   -595.0
730000161   -708.0
730010001   -736.0
730026000   -693.0
730031000   -692.0
735057548   -584.0
735059037   -734.0
760101000   -664.0
770576100   -752.0
812719000   -760.0
857632392   -792.0
8669468

Or, looking at things row-wise, we can see how a particular vessel compares with the rest of the group with respect to important features:

In [34]:
vessels[vessels.max_loa==vessels.max_loa.max()]

,num_names,names,sov,flag,flag_type,num_loas,loa,max_loa,num_types,type
mmsi,,,,,,,,,,
354092000,1,Msc Laura,N,Panama (Republic of),Foreign,3,300.0/301.0/812.0,812.0,1,Cargo


In [35]:
recs = vessels[['num_names','num_loas', 'max_loa', 'num_types']]
diff = recs - recs.loc[354092000]
diff[:10]

,num_names,num_loas,max_loa,num_types
mmsi,,,,
1,7.0,4.0,-656.0,3.0
9,2.0,-1.0,-750.0,1.0
21,0.0,-2.0,-604.0,0.0
74,1.0,-2.0,-657.0,0.0
103,2.0,-1.0,-657.0,1.0
310,0.0,-2.0,-765.0,0.0
3011,0.0,-2.0,-652.0,0.0
4731,0.0,-2.0,-782.0,0.0
15151,1.0,-1.0,-637.0,0.0


We can also apply functions to each column or row of a `DataFrame`

In [36]:
recs.apply(np.median)

num_names      1.0
num_loas       1.0
max_loa      183.0
num_types      1.0
dtype: float64

In [37]:
def range_calc(x):
    return x.max() - x.min()

In [38]:
recs.apply(range_calc)

num_names    167.0
num_loas      83.0
max_loa      812.0
num_types     13.0
dtype: float64

## Sorting and Ranking

Pandas objects include methods for re-ordering data.

In [39]:
segments.sort_index().head()

,mmsi,name,transit,segment,seg_length,avg_sog,min_sog,max_sog,pdgt10,st_time,end_time,year
0,1,Us Govt Ves,1,1,5.1,13.2,9.2,14.5,96.5,2009-02-10 16:03:00,2/10/09 16:27,2009
1,1,Dredge Capt Frank,1,1,13.5,18.6,10.4,20.6,100.0,2009-04-06 14:31:00,4/6/09 15:20,2009
2,1,Us Gov Vessel,1,1,4.3,16.2,10.3,20.5,100.0,2009-04-06 14:36:00,4/6/09 14:55,2009
3,1,Us Gov Vessel,2,1,9.2,15.4,14.5,16.1,100.0,2009-04-10 17:58:00,4/10/09 18:34,2009
4,1,Dredge Capt Frank,2,1,9.2,15.4,14.6,16.2,100.0,2009-04-10 17:59:00,4/10/09 18:35,2009


In [40]:
segments.sort_index(ascending=False).head()

,mmsi,name,transit,segment,seg_length,avg_sog,min_sog,max_sog,pdgt10,st_time,end_time,year
262525,999999999,Triple Attraction,8,1,19.8,18.6,16.1,19.5,100.0,2010-06-18 10:19:00,6/18/10 11:22,2010
262524,999999999,Triple Attraction,7,1,31.5,14.2,13.4,15.1,100.0,2010-06-18 02:52:00,6/18/10 5:03,2010
262523,999999999,Triple Attraction,6,1,17.4,17.0,14.7,18.4,100.0,2010-06-17 19:16:00,6/17/10 20:17,2010
262522,999999999,Triple Attraction,4,1,18.7,19.2,18.4,19.9,100.0,2010-06-15 21:32:00,6/15/10 22:29,2010
262521,999999999,Triple Attraction,3,1,5.3,20.0,19.6,20.4,100.0,2010-06-15 12:49:00,6/15/10 13:05,2010


Try sorting the **columns** instead of the rows, in ascending order:

In [41]:
segments.sort_index(axis=1).head()

,avg_sog,end_time,max_sog,min_sog,mmsi,name,pdgt10,seg_length,segment,st_time,transit,year
0,13.2,2/10/09 16:27,14.5,9.2,1,Us Govt Ves,96.5,5.1,1,2009-02-10 16:03:00,1,2009
1,18.6,4/6/09 15:20,20.6,10.4,1,Dredge Capt Frank,100.0,13.5,1,2009-04-06 14:31:00,1,2009
2,16.2,4/6/09 14:55,20.5,10.3,1,Us Gov Vessel,100.0,4.3,1,2009-04-06 14:36:00,1,2009
3,15.4,4/10/09 18:34,16.1,14.5,1,Us Gov Vessel,100.0,9.2,1,2009-04-10 17:58:00,2,2009
4,15.4,4/10/09 18:35,16.2,14.6,1,Dredge Capt Frank,100.0,9.2,1,2009-04-10 17:59:00,2,2009


We can also use `sort_values` to sort a `Series` by value, rather than by label.

In [42]:
segments.seg_length.sort_values(ascending=False).head(10)

181047    1882.0
181108    1691.0
149218    1675.3
49709     1647.8
149236    1644.2
50604     1557.0
106374    1517.3
140513    1502.4
140555    1491.6
50665     1489.2
Name: seg_length, dtype: float64

For a `DataFrame`, we can sort according to the values of one or more columns using the `by` argument of `sort_values`:

In [43]:
segments[['avg_sog','max_sog','min_sog']].sort_values(ascending=[False,True], 
                                           by=['max_sog', 'min_sog']).head(10)

,avg_sog,max_sog,min_sog
82222,1.5,49.9,0.0
167724,22.3,49.9,0.0
215785,2.8,49.9,0.0
119381,6.6,49.9,0.1
12479,12.2,49.9,3.5
119476,1.9,49.8,0.0
160406,9.9,49.8,0.0
162241,1.9,49.8,0.0
146139,9.6,49.8,0.1
160395,10.1,49.8,0.1


A related method `rank` does not re-arrange data, but instead returns an index that ranks each value relative to others in the Series.

In [44]:
segments.avg_sog.rank()

0         204327.5
1         255825.5
2         243231.5
3         235514.0
4         235514.0
5         262505.0
6         262506.5
7          41350.5
8          77624.0
9          65892.0
10          1122.5
11         63776.5
12         45795.5
13         86648.5
14          2122.5
15         13314.0
16         37209.5
17        212583.0
18        112262.5
19        199124.0
20        245654.5
21        127636.5
22        143948.5
23        207656.0
24        239600.5
25         65892.0
26        136465.0
27         90087.0
28        195562.5
29        140367.5
            ...   
262496     23629.0
262497     23629.0
262498     24320.5
262499     25795.0
262500     31198.5
262501     33486.5
262502     25039.5
262503     30133.0
262504     34668.0
262505     19816.5
262506     18617.0
262507     26591.0
262508     27400.0
262509     20425.0
262510     35900.0
262511     12804.5
262512     25039.5
262513     15288.5
262514     18039.5
262515     28256.0
262516     25795.0
262517     1

Ties are assigned the mean value of the tied ranks, which may result in decimal values.

In [45]:
pd.Series([100,100]).rank()

0    1.5
1    1.5
dtype: float64

Alternatively, you can break ties via one of several methods, such as by the order in which they occur in the dataset:

In [46]:
segments.avg_sog.rank(method='first').sort_values()

663            1.0
664            2.0
665            3.0
666            4.0
754            5.0
756            6.0
788            7.0
795            8.0
800            9.0
805           10.0
807           11.0
808           12.0
814           13.0
817           14.0
818           15.0
1113          16.0
1151          17.0
1152          18.0
1156          19.0
1186          20.0
1195          21.0
1227          22.0
1270          23.0
1358          24.0
2434          25.0
2436          26.0
2437          27.0
2440          28.0
2441          29.0
2443          30.0
            ...   
133971    262497.0
193181    262498.0
179881    262499.0
179882    262500.0
73207     262501.0
73225     262502.0
75990     262503.0
133963    262504.0
5         262505.0
6         262506.0
165993    262507.0
75731     262508.0
73223     262509.0
75562     262510.0
73216     262511.0
73224     262512.0
1425      262513.0
74280     262514.0
178394    262515.0
179152    262516.0
73226     262517.0
75989     26

Calling the `DataFrame`'s `rank` method results in the ranks of all columns:

In [47]:
segments.rank(ascending=False).head()

,mmsi,name,transit,segment,seg_length,avg_sog,min_sog,max_sog,pdgt10,st_time,end_time,year
0,262522.5,14015.0,256508.0,142468.5,254075.0,58199.5,62367.5,92016.5,55855.0,245583.0,176864.5,220840.5
1,262522.5,194589.5,256508.0,142468.5,237380.5,6701.5,32270.5,13177.0,19842.0,236699.5,119561.0,220840.5
2,262522.5,14047.5,256508.0,142468.5,255526.5,19295.5,33803.5,13784.5,19842.0,236696.0,119562.0,220840.5
3,262522.5,14047.5,244908.5,142468.5,247002.0,27013.0,7614.0,61125.0,19842.0,236013.0,137215.0,220840.5
4,262522.5,194589.5,244908.5,142468.5,247002.0,27013.0,7372.0,59654.0,19842.0,236012.0,137214.0,220840.5


## Hierarchical indexing

While the `mmsi` (Maritime Mobile Service Identity) field represents a unique identifier in the `vessels` table, it does not in the `segments` table.

We can use hierarchical indexing to build a **unique index** that may be more meaningful than the ordered set of integers that are given as a default index.

In [75]:
segments_hier = segments.set_index(['mmsi', 'name', 'year', 'transit', 'segment']).sort_index(ascending=True)
segments_hier.head(10)

seg_length  avg_sog  min_sog  \
mmsi name              year transit segment                                 
1    Bil Holman Dredge 2010 1       1              17.4     34.3     33.9   
                                    2              76.0     34.4     33.8   
     Dredge Capt Frank 2009 1       1              13.5     18.6     10.4   
                            2       1               9.2     15.4     14.6   
     S.d. Gumel        2011 1       1              13.7      6.9      3.0   
     Us Gov Vessel     2009 1       1               4.3     16.2     10.3   
                            2       1               9.2     15.4     14.5   
     Us Govt Ves       2009 1       1               5.1     13.2      9.2   
9    Shearwater        2010 4       1              11.6      8.8      7.9   
                            8       1              11.6      8.3      7.4   

                                             max_sog  pdgt10  \
mmsi name              year transit segment                    
1    Bil Holman Dredge 2010 1       1           35.0   100.0   
                                    2           34.9   100.0   
     Dredge Capt Frank 2009 1       1           20.6   100.0   
                            2       1           16.2   100.0   
     S.d. Gumel        2011 1       1           14.0    38.2   
     Us Gov Vessel     2009 1       1           20.5   100.0   
                            2       1           16.1   100.0   
     Us Govt Ves       2009 1       1           14.5    96.5   
9    Shearwater        2010 4       1           10.3     0.0   
                            8       1            9.2     0.0   

                                                        st_time       end_time  
mmsi name              year transit segment                                     
1    Bil Holman Dredge 2010 1       1       2010-03-20 16:06:00  3/20/10 16:36  
                                    2       2010-03-20 18:05:00  3/20/10 18:48  
     Dredge Capt Frank 2009 1       1       2009-04-06 14:31:00   4/6/09 15:20  
                            2       1       2009-04-10 17:59:00  4/10/09 18:35  
     S.d. Gumel        2011 1       1       2011-05-04 11:28:00   5/4/11 13:30  
     Us Gov Vessel     2009 1       1       2009-04-06 14:36:00   4/6/09 14:55  
                            2       1       2009-04-10 17:58:00  4/10/09 18:34  
     Us Govt Ves       2009 1       1       2009-02-10 16:03:00  2/10/09 16:27  
9    Shearwater        2010 4       1       2010-06-05 11:23:00   6/5/10 12:44  
                            8       1       2010-06-08 11:03:00   6/8/10 12:29

This index is a `MultiIndex` object that consists of a sequence of tuples, the elements of which is some combination of the three columns used to create the index. Where there are multiple repeated values, Pandas does not print the repeats, making it easy to identify groups of values.

In [76]:
segments_hier.index.is_unique

True

Try using this hierarchical index to retrieve the first segment of the tenth transit of the *Sentinel* (mmsi=366766980):

In [77]:
segments_hier.loc[(366766980, 'Sentinel', 2009, 10, 1)]

seg_length                   45.5
avg_sog                       9.8
min_sog                         6
max_sog                      10.4
pdgt10                       37.5
st_time       2009-01-23 07:08:00
end_time            1/23/09 10:49
Name: (366766980, Sentinel, 2009, 10, 1), dtype: object

With a hierachical index, we can select subsets of the data based on a *partial* index:

In [78]:
segments.loc[9]

mmsi                            9
name                   Shearwater
transit                         8
segment                         1
seg_length                   11.6
avg_sog                       8.3
min_sog                       7.4
max_sog                       9.2
pdgt10                          0
st_time       2010-06-08 11:03:00
end_time             6/8/10 12:29
year                         2010
Name: 9, dtype: object

Hierarchical indices can be created on either or both axes. Here is a trivial example:

In [79]:
frame = pd.DataFrame(np.arange(12).reshape(( 4, 3)), 
                  index =[['a', 'a', 'b', 'b'], [1, 2, 1, 2]], 
                  columns =[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])

frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

If you want to get fancy, both the row and column indices themselves can be given names:

In [80]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

With this, we can do all sorts of custom indexing:

In [81]:
frame.loc['a', 'Ohio']

color,Green,Red
key2,,
1,0,1
2,3,4


#### Exercise: Try retrieving the value corresponding to `b2` in `Colorado`:

In [82]:
# Write your answer here

Additionally, the order of the set of indices in a hierarchical `MultiIndex` can be changed by swapping them pairwise:

In [83]:
segments_hier.swaplevel('mmsi', 'name').head()

seg_length  avg_sog  min_sog  \
name              mmsi year transit segment                                 
Bil Holman Dredge 1    2010 1       1              17.4     34.3     33.9   
                                    2              76.0     34.4     33.8   
Dredge Capt Frank 1    2009 1       1              13.5     18.6     10.4   
                            2       1               9.2     15.4     14.6   
S.d. Gumel        1    2011 1       1              13.7      6.9      3.0   

                                             max_sog  pdgt10  \
name              mmsi year transit segment                    
Bil Holman Dredge 1    2010 1       1           35.0   100.0   
                                    2           34.9   100.0   
Dredge Capt Frank 1    2009 1       1           20.6   100.0   
                            2       1           16.2   100.0   
S.d. Gumel        1    2011 1       1           14.0    38.2   

                                                        st_time       end_time  
name              mmsi year transit segment                                     
Bil Holman Dredge 1    2010 1       1       2010-03-20 16:06:00  3/20/10 16:36  
                                    2       2010-03-20 18:05:00  3/20/10 18:48  
Dredge Capt Frank 1    2009 1       1       2009-04-06 14:31:00   4/6/09 15:20  
                            2       1       2009-04-10 17:59:00  4/10/09 18:35  
S.d. Gumel        1    2011 1       1       2011-05-04 11:28:00   5/4/11 13:30

In [84]:
segments_hier.swaplevel('mmsi', 'name').loc['Sentinel'].head()

seg_length  avg_sog  min_sog  max_sog  pdgt10  \
mmsi      year transit segment                                                  
366766980 2008 1       1              44.6      6.6      4.9      9.3     0.0   
               2       1              33.6      8.2      4.6      8.9     0.0   
               3       1              27.0      6.7      2.8     11.3    10.0   
               4       1              35.4     10.0      6.7     10.4    69.1   
               5       1             100.9      5.2      2.6      8.9     0.0   

                                           st_time        end_time  
mmsi      year transit segment                                      
366766980 2008 1       1       2008-11-10 05:00:00  11/10/08 11:49  
               2       1       2008-11-12 09:53:00  11/12/08 14:00  
               3       1       2008-11-24 07:02:00  11/24/08 11:01  
               4       1       2008-12-05 20:20:00   12/5/08 23:53  
               5       1       2008-12-17 00:52:00  12/17/08 20:18

Alternately, one can index based on an arbitrary level using the `xs` (cross-section) method:

In [86]:
segments_hier.xs('Sentinel', level='name').head()

seg_length  avg_sog  min_sog  max_sog  pdgt10  \
mmsi      year transit segment                                                  
366766980 2008 1       1              44.6      6.6      4.9      9.3     0.0   
               2       1              33.6      8.2      4.6      8.9     0.0   
               3       1              27.0      6.7      2.8     11.3    10.0   
               4       1              35.4     10.0      6.7     10.4    69.1   
               5       1             100.9      5.2      2.6      8.9     0.0   

                                           st_time        end_time  
mmsi      year transit segment                                      
366766980 2008 1       1       2008-11-10 05:00:00  11/10/08 11:49  
               2       1       2008-11-12 09:53:00  11/12/08 14:00  
               3       1       2008-11-24 07:02:00  11/24/08 11:01  
               4       1       2008-12-05 20:20:00   12/5/08 23:53  
               5       1       2008-12-17 00:52:00  12/17/08 20:18

Perhaps the easiest way to extract data from arbitrary levels of a `MultiIndex` is to use the `IndexSlice` function. For example, if we want the first two transits from all trips in 2009 and 2010:

In [97]:
segments_hier.loc[pd.IndexSlice[:, :, 2009:2010, [1,2]], 'seg_length'].head(12)

mmsi  name               year  transit  segment
1     Bil Holman Dredge  2010  1        1          17.4
                                        2          76.0
      Dredge Capt Frank  2009  1        1          13.5
                               2        1           9.2
      Us Gov Vessel      2009  1        1           4.3
                               2        1           9.2
      Us Govt Ves        2009  1        1           5.1
74    Mcfaul             2009  1        2           1.4
      Sarah Bell         2009  1        1           7.4
103   Ron G              2009  1        1          87.5
                                        3           1.4
                               2        1          30.7
Name: seg_length, dtype: float64

The `:` tells pandas to include the entire level. What `IndexSlice` does is generate a Python `slice` object that represents what is required from each level. So, the above is equivalent to:

    (slice(None, None, None), slice(None, None, None), [2009, 2010], [1, 2])

Data can also be sorted by any index level, using `sort_index` with the `level` argument appropriately specified:

In [99]:
segments.sort_index(level='max_sog', ascending=False).head()

,mmsi,name,transit,segment,seg_length,avg_sog,min_sog,max_sog,pdgt10,st_time,end_time,year
262525,999999999,Triple Attraction,8,1,19.8,18.6,16.1,19.5,100.0,2010-06-18 10:19:00,6/18/10 11:22,2010
262524,999999999,Triple Attraction,7,1,31.5,14.2,13.4,15.1,100.0,2010-06-18 02:52:00,6/18/10 5:03,2010
262523,999999999,Triple Attraction,6,1,17.4,17.0,14.7,18.4,100.0,2010-06-17 19:16:00,6/17/10 20:17,2010
262522,999999999,Triple Attraction,4,1,18.7,19.2,18.4,19.9,100.0,2010-06-15 21:32:00,6/15/10 22:29,2010
262521,999999999,Triple Attraction,3,1,5.3,20.0,19.6,20.4,100.0,2010-06-15 12:49:00,6/15/10 13:05,2010


## Missing data

The occurence of missing data is so prevalent that it pays to use tools like Pandas, which seamlessly integrates missing data handling so that it can be dealt with easily, and in the manner required by the analysis at hand.

Missing data are represented in `Series` and `DataFrame` objects by the `NaN` floating point value. However, `None` is also treated as missing, since it is commonly used as such in other contexts (*e.g.* NumPy).

In [ ]:
foo = pd.Series([np.nan, -3, None, 'foobar'])
foo

In [ ]:
foo.isnull()

To illustrate how to deal with missing values in pandas, we will import an educational database, which reports test scores for children with hearing disabilities, along with associated covariates, several of which are missing.

In [ ]:
testing = pd.read_csv('../data/test_scores.csv', index_col=0)
testing.head(10)

Missing values may be dropped or indexed out:

By default, `dropna` drops entire rows in which one or more values are missing.

In [ ]:
testing.dropna().head(10)

A similar result can be obtained via indexing.

In [ ]:
testing.isnull().head()

In [ ]:
testing[testing.notnull()].head()

This can be overridden by passing the `how='all'` argument, which only drops a row when every field is a missing value.

In [ ]:
testing.dropna(how='all')

This can be customized further by specifying how many values need to be present before a row is dropped via the `thresh` argument.

In [ ]:
testing.dropna(thresh=10)

This is typically used in time series applications, where there are repeated measurements that are incomplete for some subjects.

### Exercise

Try using the `axis` argument to drop columns with missing values:

In [ ]:
# Write your answer here

Rather than omitting missing data from an analysis, in some cases it may be suitable to fill the missing value in, either with a default value (such as zero) or a value that is either imputed or carried forward/backward from similar data points. We can do this programmatically in Pandas with the `fillna` argument.

In [ ]:
testing.fillna(0).head(10)

In [ ]:
testing.fillna({'family_inv': 0, 'mother_hs':1}).head(10)

Notice that `fillna` by default returns a new object with the desired filling behavior, rather than changing the `Series` or  `DataFrame` in place (**in general, we like to do this, by the way!**).

Missing values can also be interpolated, using any one of a variety of methods:

In [ ]:
testing.fillna(method='bfill').head(10)

## Data summarization

We often wish to summarize data in `Series` or `DataFrame` objects, so that they can more easily be understood or compared with similar data. The NumPy package contains several functions that are useful here, but several summarization or reduction methods are built into Pandas data structures.

In [ ]:
testing.sum()

Clearly, `sum` is more meaningful for some columns than others. For methods like `mean` for which application to string variables is not just meaningless, but impossible, these columns are automatically exculded:

In [ ]:
testing.mean()

The important difference between NumPy's functions and Pandas' methods is that the latter have built-in support for handling missing data.

Sometimes we may not want to ignore missing values, and allow the `nan` to propagate.

In [ ]:
testing.mean(skipna=False)

Passing `axis=1` will summarize over rows instead of columns, which only makes sense in certain situations.

In [ ]:
nonenglish_nonwhite_withHS = testing[['non_english', 'mother_hs', 'non_white']].prod(axis=1)

nonenglish_nonwhite_withHS

A useful summarization that gives a quick snapshot of multiple statistics for a `Series` or `DataFrame` is `describe`:

In [ ]:
testing.describe()

`describe` can detect non-numeric data and sometimes yield useful information about it.

In [ ]:
testing.non_english.describe()

We can also calculate summary statistics *across* multiple columns, for example, correlation and covariance.

$$cov(x,y) = \sum_i (x_i - \bar{x})(y_i - \bar{y})$$

In [ ]:
testing.score.cov(testing.age_test)

$$corr(x,y) = \frac{cov(x,y)}{(n-1)s_x s_y} = \frac{\sum_i (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum_i (x_i - \bar{x})^2 \sum_i (y_i - \bar{y})^2}}$$

In [ ]:
testing.score.corr(testing.age_test)

Try running `corr` on the entire `testing` DataFrame to see what is returned:

In [ ]:
# Write answer here

If we have a `DataFrame` with a hierarchical index (or indices), summary statistics can be applied with respect to any of the index levels:

In [ ]:
segments_hier.head()

In [ ]:
segments_hier.sum(level='transit').seg_length

## Writing Data to Files

As well as being able to read several data input formats, Pandas can also export data to a variety of storage formats. We will bring your attention to just a couple of these.

In [ ]:
testing.to_csv("testing.csv")

The `to_csv` method writes a `DataFrame` to a comma-separated values (csv) file. You can specify custom delimiters (via `sep` argument), how missing values are written (via `na_rep` argument), whether the index is writen (via `index` argument), whether the header is included (via `header` argument), among other options.

An efficient way of storing data to disk is in binary format. Pandas supports this using Python’s built-in pickle serialization.

In [ ]:
testing.to_pickle("testing_pickle")

The complement to `to_pickle` is the `read_pickle` function, which restores the pickle to a `DataFrame` or `Series`:

In [ ]:
pd.read_pickle("testing_pickle").head()

As Wes warns in his book, it is recommended that binary storage of data via pickle only be used as a temporary storage format, in situations where speed is relevant. This is because there is no guarantee that the pickle format will not change with future versions of Python.

## References

[Python for Data Analysis](http://shop.oreilly.com/product/0636920023784.do) Wes McKinney